<a href="https://colab.research.google.com/github/mashine01/JournalismAI/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)
  Using cached peft-0.4.0-py3-none-any.whl (72 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.5 MB/s eta 0:00:00
  Using cached trl-0.4.7-py3-none-any.whl (77 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.3 MB/s eta 0:00:00
  Using cached datasets-2.16.0-py3-none-any.whl (507 kB)
  Using cached pyarrow_hotfix-0.6-py3-none-any.whl (7.9 kB)
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2

In [ ]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer
import pandas as pd

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
base_model = "NousResearch/Llama-2-7b-hf"

In [ ]:
def loadModel(model_name, bnb_config):

    n_gpus = torch.cuda.device_count()
    max_memory = f'{20000}MB'

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    # Load model tokenizer with the user authentication token
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)

    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [ ]:
# Model from Hugging Face hub
model_name = "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_use_double_quant = True,
        bnb_4bit_compute_dtype = torch.bfloat16,
        bnb_4bit_quant_type = "nf4",
        )
model, tokenizer = loadModel(model_name, bnb_config)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
df1=pd.read_csv('news.csv')
# Add a new column with the same value for all rows
df1['Instruction']=f'Generate an Article for the category of sports \n weather'

# Save the DataFrame back to the CSV file
df1.to_csv("news.csv", index=False)

In [ ]:
# Load dataset
dataset = load_dataset("csv", data_files = "news.csv", split = "train")

print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Generating train split: 0 examples [00:00, ? examples/s]

Number of prompts: 2087
Column names are: ['Article', 'Category', 'Link', 'Instruction']


In [ ]:
print(dataset[randrange(len(dataset))])

{'Article': 'The West Coast will luck out this Halloween. Temperatures will be near normal, from the 50s to the mid 70s, and the region will also dodge rain.', 'Category': 'Weather', 'Link': 'https://edition.cnn.com/2023/10/27/weather/halloween-weather-trick-or-treat-cold-climate/index.html', 'Instruction': 'Generate an Article for the category of sports \n weather'}


In [ ]:
def promptFormat(sample):

    # Initialize static strings for the prompt template
    promptTemplate = """You are a news editor and your task is to write article. So write an article on the category of {category} of {city_name} using the data output from the api {api_data}."""
    Instruction = "Instruction:"
    Article = "Article:"
    Category = "Category:"
    ending = "End"

    # Combine a prompt with the static strings
    blurb = f"{promptTemplate}"
    taskInstruction = f"{Instruction}\n{sample['Instruction']}"
    inputArticle = f"{Article}\n{sample['Article']}" if sample["Article"] else None
    article_topic = f"{Category}\n{sample['Category']}"
    end = f"{ending}"

    # Create a list of prompt template elements
    parts = [part for part in [blurb, taskInstruction, inputArticle, article_topic, end] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["text"] = formatted_prompt

    return sample

In [ ]:
def preprocess_batch(batch, tokenizer, max_length):
    return tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

In [ ]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):

    dataset = dataset.map(promptFormat)
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ["Article", "Category", "Link", "text"],
    )

    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [ ]:
# Random seed
seed = 10

maxLength = 4096
preprocessedData = preprocess_dataset(tokenizer, maxLength, seed, dataset)

print(preprocessedData)

print(preprocessedData[0])

Map:   0%|          | 0/2087 [00:00<?, ? examples/s]

Map:   0%|          | 0/2087 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2087 [00:00<?, ? examples/s]

Dataset({
    features: ['Instruction', 'input_ids', 'attention_mask'],
    num_rows: 2055
})
{'Instruction': 'Generate an Article for the category of sports \n weather', 'input_ids': [1, 887, 526, 263, 9763, 6920, 322, 596, 3414, 338, 304, 2436, 4274, 29889, 1105, 2436, 385, 4274, 373, 278, 7663, 310, 426, 7320, 29913, 310, 426, 12690, 29918, 978, 29913, 773, 278, 848, 1962, 515, 278, 7882, 426, 2754, 29918, 1272, 1836, 13, 1678, 450, 4274, 881, 367, 367, 3309, 29891, 29892, 12242, 363, 472, 280, 579, 29871, 29947, 29900, 29900, 3838, 29889, 13, 13, 3379, 4080, 29901, 13, 5631, 403, 385, 21746, 363, 278, 7663, 310, 14717, 29871, 13, 14826, 13, 13, 9986, 2512, 29901, 13, 29943, 417, 397, 6505, 267, 393, 15201, 14746, 4822, 13267, 354, 11530, 13813, 505, 9698, 541, 263, 6505, 9488, 297, 2058, 363, 8886, 29871, 29896, 29889, 29945, 7284, 2305, 3412, 18632, 13813, 1549, 278, 11005, 29889, 5057, 8805, 6655, 29879, 526, 884, 297, 2058, 4822, 14841, 322, 18632, 13813, 29892, 411, 29871, 2995

In [ ]:
def peftConfig(r, lora_alpha, target_modules, lora_dropout, bias, task_type):

    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

In [ ]:
def find_all_linear_names(model):

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [ ]:
def print_trainable_parameters(model, use_4bit = False):

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [ ]:
def fine_tune(model, tokenizer, dataset, lora_r, lora_alpha, lora_dropout, bias, task_type, per_device_train_batch_size, gradient_accumulation_steps, warmup_steps, max_steps, learning_rate, fp16, logging_steps, output_dir, optim):

    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    target_modules = find_all_linear_names(model)
    peft_config = peftConfig(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Training parameters
    trainer = Trainer(
        model = model,
        train_dataset = dataset,
        args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
        ),
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )

    model.config.use_cache = False

    do_train = True

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    del model
    del trainer
    torch.cuda.empty_cache()

In [ ]:
# LoRA attention dimension
lora_r = 16
# Alpha parameter for LoRA scaling
lora_alpha = 64
# Dropout probability for LoRA layers
lora_dropout = 0.1
# Bias
bias = "none"
# Task type
task_type = "CAUSAL_LM"
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"
# Batch size per GPU for training
per_device_train_batch_size = 1
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
# Optimizer to use
optim = "paged_adamw_32bit"
# Number of training steps
max_steps = 100
# Linear warmup steps from 0 to learning_rate
warmup_steps = 2
# Enable fp16/bf16 training
fp16 = True
# Log every step
logging_steps = 1

In [ ]:
fine_tune(model,
          tokenizer,
          preprocessedData,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim)

LoRA module names: ['q_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj', 'k_proj', 'v_proj']
All Parameters: 3,540,389,888 || Trainable Parameters: 39,976,960 || Trainable Parameters %: 1.1291682911958425


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,2.900300
2,2.771300
3,2.418500
4,1.722000
5,1.419300
6,0.921000
7,0.820100
8,0.792600
9,0.650500
10,0.761200


***** train metrics *****
  epoch                    =       0.19
  total_flos               =  1230659GF
  train_loss               =     0.7483
  train_runtime            = 0:12:48.46
  train_samples_per_second =      0.521
  train_steps_per_second   =       0.13
{'train_runtime': 768.4685, 'train_samples_per_second': 0.521, 'train_steps_per_second': 0.13, 'total_flos': 1321410701574144.0, 'train_loss': 0.7483430030941963, 'epoch': 0.19}


In [ ]:
# Load fine-tuned weights
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, torch_dtype = torch.bfloat16)
# Merge the LoRA layers with the base model
model = model.merge_and_unload()

# Save fine-tuned model at a new location
output_merged_dir = "results/mash01/final_checkpoint"
os.makedirs(output_merged_dir, exist_ok = True)
model.save_pretrained(output_merged_dir, safe_serialization = True)

# Save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)

# Fine-tuned model name on Hugging Face Hub
new_model = "mash01/llama2_article2"

# Push fine-tuned model and tokenizer to Hugging Face Hub
new_model = "llama2_articles" # Fine-tuned model
model.push_to_hub(new_model, use_auth_token = True)
tokenizer.push_to_hub(new_model, use_auth_token = True)